In [17]:
from anaconda_anon_usage.tokens import client_token
from telethon.sync import TelegramClient as TC
from telethon import TelegramClient
from telethon.tl.functions.channels import JoinChannelRequest
import telethon
import json
from PIL import Image
from io import BytesIO
import networkx as nx

In [2]:
credentials = json.load(open('../../keys/telegram_key.json'))
client_id = credentials['api_id']
client_secret = credentials['api_hash']

In [3]:
# Costruisco oggetto TelegramClient

client = TelegramClient('anon', client_id, client_secret)

In [16]:
async with client:
    user = await client.get_me()

print(user.first_name)

Filippo


In [31]:
async with client:
    #channel = await client.get_entity('JakiDale')
    channel = await client.get_entity(1163643200)

    #await client(JoinChannelRequest(channel.id))

print(channel.stringify())

Channel(
	id=1163643200,
	title='Prodigeek Official',
	photo=ChatPhoto(
		photo_id=6023591964085171127,
		dc_id=4,
		has_video=False,
		stripped_thumb=b'\x01\x08\x08\xd5\xfd\xe7\x9aq\xf7}\xe8\xa2\x8a\x05c'
	),
	date=datetime.datetime(2025, 3, 14, 9, 5, tzinfo=datetime.timezone.utc),
	creator=False,
	left=False,
	broadcast=False,
	verified=False,
	megagroup=True,
	restricted=False,
	signatures=False,
	min=False,
	scam=False,
	has_link=True,
	has_geo=False,
	slowmode_enabled=False,
	call_active=False,
	call_not_empty=False,
	fake=False,
	gigagroup=False,
	noforwards=False,
	join_to_send=True,
	join_request=False,
	forum=False,
	stories_hidden=False,
	stories_hidden_min=False,
	stories_unavailable=True,
	signature_profiles=False,
	access_hash=384215503251515819,
	username=None,
	restriction_reason=[
	],
	admin_rights=None,
	banned_rights=None,
	default_banned_rights=ChatBannedRights(
		until_date=datetime.datetime(2038, 1, 19, 3, 14, 7, tzinfo=datetime.timezone.utc),
		view_messages=False

In [34]:
async with client:
    dialogs = await client.get_dialogs()

    for dialog in dialogs:
        if 'prodigeek' in dialog.title.lower():
            dialog_id = dialog.id


print(f"Conversazione con Prodigeek trovata con id {dialog_id}")



Conversazione con Prodigeek trovata con id -1001163643200


In [58]:
# Leggiamo 3000 messaggi

group_messages = dict()
async with client:
    messages = client.iter_messages(entity=channel,limit=3000)
    async for message in messages:
        group_messages[message.id] = message





In [69]:
message_sample = group_messages[list(group_messages.keys())[2]]

print(len(group_messages))
print(message_sample.stringify())

3000
Message(
	id=8284605,
	peer_id=PeerChannel(
		channel_id=1163643200
	),
	date=datetime.datetime(2025, 3, 14, 9, 54, 14, tzinfo=datetime.timezone.utc),
	message='Comunque per non rischiare mi butto sulla b850 gigabyte aorus élite costicchia un po’ di più rispetto alla tomahawk ma vabbè',
	out=False,
	mentioned=False,
	media_unread=False,
	silent=False,
	post=False,
	from_scheduled=False,
	legacy=False,
	edit_hide=False,
	pinned=False,
	noforwards=False,
	invert_media=False,
	offline=False,
	video_processing_pending=False,
	from_id=PeerUser(
		user_id=765774617
	),
	from_boosts_applied=None,
	saved_peer_id=None,
	fwd_from=None,
	via_bot_id=None,
	via_business_bot_id=None,
	reply_to=None,
	media=None,
	reply_markup=None,
	entities=[
	],
	views=None,
	forwards=None,
	replies=MessageReplies(
		replies=0,
		replies_pts=9469750,
		comments=False,
		recent_repliers=[
		],
		channel_id=None,
		max_id=None,
		read_max_id=None
	),
	edit_date=None,
	post_author=None,
	grouped_id=None,
	reacti

In [88]:
# Find all unique users who sent those messages

active_users = set()

for message_id, message in group_messages.items():
    user = message.from_id
    if user is not None and 'user_id' in user.__dict__:
        active_users.add(user.user_id)

print(len(active_users))

327


In [91]:
# Find non-reply messages

non_reply_messages = set()

for message_id, message in group_messages.items():
    if message.reply_to is None:
        non_reply_messages.add(message_id)

print(len(non_reply_messages))

887


In [93]:
# Build a list of (Reply-Message, Source-Message)

replies_list = list()

for message_id, message in group_messages.items():
    if message_id in non_reply_messages:
        continue

    assert message.reply_to is not None

    replied_to_message_id = message.reply_to.reply_to_msg_id

    replies_list.append((message_id, replied_to_message_id))

In [94]:
len(replies_list)

2113

In [142]:
# Build the Network from the Edge List

def author_id_by_message_id(message_id):
    if message_id not in group_messages:
        return None
    message = group_messages[message_id]
    user = message.from_id
    if user is not None and 'user_id' in user.__dict__:
        return user.user_id
    return None

G = nx.DiGraph()
for b, a in replies_list:
    A, B = author_id_by_message_id(a), author_id_by_message_id(b)
    if A and B:
        G.add_edge(B, A)

G.order(), G.size()

(271, 920)

In [158]:
# Add attributes

for b, a in replies_list:
    if a not in group_messages or b not in group_messages:
        continue
    msgA, msgB = group_messages[a], group_messages[b]
    A, B = author_id_by_message_id(a), author_id_by_message_id(b)
    if A and B:
        if 'messaggi' not in G[B][A]:
            G[B][A]['messaggi'] = list()

        G[B][A]['messaggi'].append((msgB.message, msgB.date))


In [159]:
nx.write_gexf(
    G=G,
    path='../../data/prodigeek_telegram.gexf'
)


ValueError: not enough values to unpack (expected 3, got 2)

In [153]:
for node, data in G.nodes(data=True):
    print(node, data)  # Check attributes

90865968 {}
321622456 {}
572151306 {}
6925718407 {}
885265684 {}
6827594915 {}
909915156 {}
180449098 {}
603486804 {}
71256842 {}
765774617 {}
201818567 {}
7585388306 {}
6417678372 {}
1479991055 {}
465711300 {}
479105570 {}
1721565788 {}
7172433018 {}
688801613 {}
157914392 {}
205515236 {}
2111789119 {}
787538373 {}
5635997899 {}
641591682 {}
244688696 {}
738109516 {}
170324226 {}
172608813 {}
5965381639 {}
6397236863 {}
637577801 {}
262721749 {}
1153693273 {}
372092129 {}
7738142636 {}
155278659 {}
526410152 {}
8106971140 {}
1383223010 {}
1533967032 {}
5774394 {}
6186720734 {}
1136684968 {}
904741694 {}
135270368 {}
258690385 {}
679233748 {}
262975777 {}
140503703 {}
353595053 {}
6435592818 {}
211779187 {}
833205393 {}
359247660 {}
421483548 {}
108065640 {}
173053580 {}
848865238 {}
5518749328 {}
303982486 {}
1083488870 {}
233182657 {}
1616841009 {}
5854799531 {}
13564711 {}
8198690800 {}
552924006 {}
288813584 {}
979784049 {}
1889926526 {}
5399883108 {}
348963294 {}
78941955 {}
19351